# Day 9: Disk Fragmenter

Another push of the button leaves you in the familiar hallways of some friendly [amphipods](https://adventofcode.com/2021/day/23)! Good thing you each somehow got your own personal mini submarine. The Historians jet away in search of the Chief, mostly by driving directly into walls.

While The Historians quickly figure out how to pilot these things, you notice an amphipod in the corner struggling with his computer. He's trying to make more contiguous free space by compacting all of the files, but his program isn't working; you offer to help.

He shows you the _disk map_ (your puzzle input) he's already generated. For example:

```
2333133121414131402
```

The disk map uses a dense format to represent the layout of _files_ and _free space_ on the disk. The digits alternate between indicating the length of a file and the length of free space.

So, a disk map like `12345` would represent a one-block file, two blocks of free space, a three-block file, four blocks of free space, and then a five-block file. A disk map like `90909` would represent three nine-block files in a row (with no free space between them).

Each file on disk also has an _ID number_ based on the order of the files as they appear _before_ they are rearranged, starting with ID `0`. So, the disk map `12345` has three files: a one-block file with ID `0`, a three-block file with ID `1`, and a five-block file with ID `2`. Using one character for each block where digits are the file ID and `.` is free space, the disk map `12345` represents these individual blocks:

```
0..111....22222
```

The first example above, `2333133121414131402`, represents these individual blocks:

```
00...111...2...333.44.5555.6666.777.888899
```

The amphipod would like to _move file blocks one at a time_ from the end of the disk to the leftmost free space block (until there are no gaps remaining between file blocks). For the disk map `12345`, the process looks like this:

```
0..111....22222
02.111....2222.
022111....222..
0221112...22...
02211122..2....
022111222......
```

The first example requires a few more steps:

```
00...111...2...333.44.5555.6666.777.888899
009..111...2...333.44.5555.6666.777.88889.
0099.111...2...333.44.5555.6666.777.8888..
00998111...2...333.44.5555.6666.777.888...
009981118..2...333.44.5555.6666.777.88....
0099811188.2...333.44.5555.6666.777.8.....
009981118882...333.44.5555.6666.777.......
0099811188827..333.44.5555.6666.77........
00998111888277.333.44.5555.6666.7.........
009981118882777333.44.5555.6666...........
009981118882777333644.5555.666............
00998111888277733364465555.66.............
0099811188827773336446555566..............
```

The final step of this file-compacting process is to update the _filesystem checksum_. To calculate the checksum, add up the result of multiplying each of these blocks' position with the file ID number it contains. The leftmost block is in position `0`. If a block contains free space, skip it instead.

Continuing the first example, the first few blocks' position multiplied by its file ID number are `0 * 0 = 0`, `1 * 0 = 0`, `2 * 9 = 18`, `3 * 9 = 27`, `4 * 8 = 32`, and so on. In this example, the checksum is the sum of these, `_1928_`.

Compact the amphipod's hard drive using the process he requested. _What is the resulting filesystem checksum?_ (Be careful copy/pasting the input for this puzzle; it is a single, very long line.)

In [ ]:
$sample_data = (@'
2333133121414131402
'@).Replace("`r", '') -split "`n" 

$input_data = Get-Content .\data.txt

In [ ]:
function dostuff {
    [CmdletBinding()] Param(
        [Parameter(Mandatory=$false)]
        [switch] $sample,
        [Parameter(Mandatory=$false)]
        [switch] $part2
    )
    if($sample) {
        $data = [string]$sample_data
        $out = "With *SAMPLE* dataset, part "
    } else {
        $data = $input_data
        $out = "Part "
    }
    $out += $part2 ? '2: ' : '1: '
    
    $spaceReqd = 0
    for($x=0; $x -lt $data.Length; $x++){
        $spaceReqd += [int](' ' + $data[$x])
    }
    Write-Output "Array size reqd: $spaceReqd, using (u)int32, that'd be $([math]::round($spaceReqd*32/1Mb, 1))Mb"
    
    $disk = [int[]]::new($spaceReqd); 

    $file = $true
    $fileIndex = 0
    $blockPointer = 0
    Write-Progress -Activity "Defrag" -Status "Building disk map" -PercentComplete 0
    for($x=0; $x -lt $data.Length; $x++) {
        if($file) {
            $value = $fileIndex
            $fileIndex++
            $file = $false
        } else {
            $value = -1
            $file = $true
        }
        # the `' ' + ` is because otherwise PoSH thinks we're converting [char] (and not [string]) to [int]
        if($data[$x] -ne '0') {
            $upperBound = $blockPointer+(' ' + $data[$x])-1
            Write-Verbose "Reading pos $x, data $($data[$x]). Setting $blockPointer..$upperBound to $value"
            $blockPointer..$upperBound | ForEach-Object {$disk[$_] = $value}
            $blockPointer = $upperBound + 1
        }
        if($x % 10 -eq 0) {
            #Write-Progress -Activity "Defrag" -Status "Building disk map" -PercentComplete $x/$data.Length*10
        }
    }
    Write-Debug "disk: $(($disk -join '').Replace('-1','.'))"

    $freespacePointer = $disk.IndexOf(-1)
    for($x=$disk.count-1; $x -gt 0; $x--) {
        if($disk[$x] -eq -2) {
            Write-Verbose "Found moved block!"
            break
        } elseif($disk[$x] -eq -1) {
            Write-Verbose 'skipping moved block?'
            continue
        } else {
            Write-Debug "Moving block at $x part of file $($disk[$x]) to pos $freespacePointer. disk: $(($disk -join '').Replace('-1','.').Replace('-2','#'))"
            $disk[$freespacePointer] = $disk[$x]
            $freespacePointer = $disk.IndexOf(-1)
            $disk[$x] = -1 #mark file as moved.
            if($freespacePointer -eq -1){
                Write-Warning "No more free space!"
                break
            }
            if($freespacePointer -gt $x) {
                Write-Debug "Complete!"
                break
            }
        }
        
        if($x % 10 -eq 0) {
            $percent = (10 + ($x/$disk.count*60))
            Write-Progress -Activity "Defrag" -Status "Defragmenting ($percent%)" -PercentComplete $percent
        }
    }

    Write-Debug "disk: $(($disk -join '').Replace('-1','.').Replace('-2','#'))"

    $positon = 0
    $checksum = 0
    foreach($number in $disk) {
        if($number -lt 0) {
            break
        }
        $checksum += $number * $positon
        $positon++
        if($positon % 10 -eq 0) {
            $percent = (70 + ($x/$disk.count*30))
            Write-Progress -Activity "Defrag" -Status "Calculating checksum ($percent%)" -PercentComplete $percent
        }
    }
    $out += "disk checksum is $checksum"
    Write-Output $out
}

dostuff -sample -Debug
dostuff

Array size reqd: 42, using (u)int32, that'd be 0Mb


                                                                                                    
 Defrag                                                                                             
    Defragmenting                                                                                   
    [oooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo                            ]      
                                                                                                    

DEBUG: disk: 00...111...2...333.44.5555.6666.777.888899
DEBUG: Moving block at 41 part of file 9 to pos 2. disk: 00...111...2...333.44.5555.6666.777.888899
DEBUG: Moving block at 40 part of file 9 to pos 3. disk: 009..111...2...333.44.5555.6666.777.88889.
DEBUG: Moving block at 39 part of file 8 to pos 4. disk: 0099.111...2...333.44.5555.6666.777.8888..
DEBUG: Moving block at 38 part of file 8 to pos 8. disk: 00998111...2...333.44.5555.6666.777.888...
DEBUG: Moving block at 37 part of file 8 to pos 9. disk: 009981118..2...333.44.5555.6666.777.88....
DEBUG: Moving block at 36 part of file 8 to pos 10. disk: 0099811188.2...333.44.5555.6666.777.8.....
DEBUG: Moving block at 34 part of file 7 to pos 12. disk: 009981118882...333.44.5555.6666.777.......
DEBUG: Moving block at 33 part of file 7 to pos 13. disk: 0099811188827..333.44.5555.6666.77........
DEBUG: Moving block at 32 part of file 7 to pos 14. disk: 00998111888277.333.44.5555.6666.7.........
DEBUG: Moving block at 30 part of file 6

Error: Command cancelled.